In [39]:
import numpy as np
import tensorflow as tf
import keras as k
import nibabel as nib

COLAB = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  DATA_PATH = '/content/drive/My Drive/Machine learning/data'

else: 
  DATA_PATH = '../data'

# Prueba de cómo cargar una imagen con Nibabel

prueba_filename = DATA_PATH + '/Datos_TFG_Angel/ppNOR/MRI/m0wrp2ADNI_941_S_1203_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070801201705724_S25671_I63879.nii'
img = nib.load(prueba_filename)

image_data = img.get_fdata()

print(type(image_data))

np.mean(image_data[~np.isnan(image_data)])

# with open('/content/drive/My Drive/Machine learning/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat /content/drive/My\ Drive/Machine\ learning/foo.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'numpy.memmap'>


0.09722850863292964